In [1]:
# One time install of dependancy
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [20]:
# Install dependancies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt

from sqlalchemy import create_engine
import psycopg2
import config as creds

In [26]:
# Retreive url address from Amazon RDS and test reading of a file
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/imdb_main.csv"
imdb_main = pd.read_csv(url)
imdb_main.head()

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_Mystery,g_News,g_Reality_TV,g_Romance,g_Sci_Fi,g_Sport,g_Thriller,g_War,g_Western,director_list
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Raymond Longford']
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,NaN,NaN,...,0,0,0,0,0,0,0,0,0,['Arthur Robison']
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Jean Epstein']
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,NaN,...,0,0,0,0,0,0,0,0,0,['Cecil B. DeMille']
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,NaN,NaN,...,0,0,0,0,0,0,0,0,0,"['Edward Sedgwick', ' Buster Keaton']"


In [27]:
# Use the help function to understand the parameters that may be run with 'to_sql'
help(imdb_main.to_sql)

Help on method to_sql in module pandas.core.generic:

to_sql(name: 'str', con, schema=None, if_exists: 'str' = 'fail', index: 'bool_t' = True, index_label=None, chunksize=None, dtype=None, method=None) -> 'None' method of pandas.core.frame.DataFrame instance
    Write records stored in a DataFrame to a SQL database.
    
    Databases supported by SQLAlchemy [1]_ are supported. Tables can be
    newly created, appended to, or overwritten.
    
    Parameters
    ----------
    name : str
        Name of SQL table.
    con : sqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library. Legacy support is provided for sqlite3.Connection objects. The user
        is responsible for engine disposal and connection closure for the SQLAlchemy
        connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_.
    
    schema : str, optional
        Specify the sch

In [28]:
# use info to check the column info
imdb_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  ge

In [10]:
# run code to update capitalized letters to lowercase letters in order to write to lowercase only
# in posgres since empty database was created in advance
lowercase_columns = [col.lower() for col in imdb_main.columns]
imdb_main.columns = lowercase_columns
imdb_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  ge

In [21]:
from config import * # import all from config to get credentials and host and database names in postgress
database_string = f"postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}"
# create variable for the engine
engine = create_engine(database_string)


In [ ]:
# write imdb_kmain to postgres file that 'already exists'
imdb_main.to_sql("imdb_main", engine, if_exists = 'append', index = False, chunksize = 20)

In [12]:
# get url address from AWS 'RDS' and 'S3 bucket' and read file tmdb_main
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/tmdb_main.csv"
tmdb_main = pd.read_csv(url)
tmdb_main.head()

,budget,imdb_id,original_language,popularity,revenue,runtime,release_year,orig_lang_cd,collection,website,...,g_Crime,g_Thriller,g_Horror,g_History,g_Mystery,g_War,g_Foreign,g_Music,g_Documentary,g_Western
0,30000000,tt0114709,en,21.946943,373554033,81.0,1995,en,yes,yes,...,0,0,0,0,0,0,0,0,0,0
1,65000000,tt0113497,en,17.015539,262797249,104.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
2,0,tt0113228,en,11.712900,0,101.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0
3,16000000,tt0114885,en,3.859495,81452156,127.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
4,0,tt0113041,en,8.387519,76578911,106.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# convert uppercase to lowercase for tmdb_main file to be compatible with postres lowercase only standard
lowercase_columns = [col.lower() for col in tmdb_main.columns]
tmdb_main.columns = lowercase_columns
tmdb_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37049 entries, 0 to 37049
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             37049 non-null  int64  
 1   imdb_id            37049 non-null  object 
 2   original_language  37043 non-null  object 
 3   popularity         37049 non-null  float64
 4   revenue            37049 non-null  int64  
 5   runtime            36835 non-null  float64
 6   release_year       37049 non-null  int64  
 7   orig_lang_cd       37049 non-null  object 
 8   collection         37049 non-null  object 
 9   website            37049 non-null  object 
 10  genre_name         34934 non-null  object 
 11  g_animation        37049 non-null  int64  
 12  g_comedy           37049 non-null  int64  
 13  g_family           37049 non-null  int64  
 14  g_adventure        37049 non-null  int64  
 15  g_fantasy          37049 non-null  int64  
 16  g_romance          370

In [ ]:
tm3 = joined_tables

In [24]:
# from config import *
# database_string = f"postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}"
# engine = create_engine(database_string)
# bypassed above code as it was already read in previously
# write tmdb_main to preexisting empty database in postgres
tmdb_main.to_sql("tmdb_main", engine, if_exists = 'append', index = False, chunksize = 20)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "tmdb_main_pkey"
DETAIL:  Key (imdb_id)=(tt0114709) already exists.

[SQL: INSERT INTO tmdb_main (budget, imdb_id, original_language, popularity, revenue, runtime, release_year, orig_lang_cd, collection, website, genre_name, g_animation, g_comedy, g_family, g_adventure, g_fantasy, g_romance, g_drama, g_action, g_crime, g_thriller, g_horror, g_history, g_mystery, g_war, g_foreign, g_music, g_documentary, g_western) VALUES (%(budget)s, %(imdb_id)s, %(original_language)s, %(popularity)s, %(revenue)s, %(runtime)s, %(release_year)s, %(orig_lang_cd)s, %(collection)s, %(website)s, %(genre_name)s, %(g_animation)s, %(g_comedy)s, %(g_family)s, %(g_adventure)s, %(g_fantasy)s, %(g_romance)s, %(g_drama)s, %(g_action)s, %(g_crime)s, %(g_thriller)s, %(g_horror)s, %(g_history)s, %(g_mystery)s, %(g_war)s, %(g_foreign)s, %(g_music)s, %(g_documentary)s, %(g_western)s)]
[parameters: ({'budget': 30000000, 'imdb

In [7]:
# Write the query to join the database tables that now exist and are populated in postgres
query = '''SELECT * FROM imdb_main LEFT JOIN tmdb_main ON imdb_main.imdb_id = tmdb_main.imdb_id;'''

# Use pandas to run the query on the database, with the engine created above
joined_tables = pd.read_sql_query(query, con=engine)

In [11]:
joined_tables.columns

Index(['imdb_id', 'title', 'year', 'duration', 'country', 'language',
       'avg_vote', 'votes', 'budget', 'usa_gross_income',
       'worlwide_gross_income', 'metascore', 'reviews_from_users',
       'reviews_from_critics', 'release_year', 'genre_list', 'g_action',
       'g_adult', 'g_adventure', 'g_animation', 'g_biography', 'g_comedy',
       'g_crime', 'g_documentary', 'g_drama', 'g_family', 'g_fantasy',
       'g_history', 'g_horror', 'g_music', 'g_musical', 'g_mystery', 'g_news',
       'g_reality_tv', 'g_romance', 'g_sci_fi', 'g_sport', 'g_thriller',
       'g_war', 'g_western', 'director_list', 'budget', 'imdb_id',
       'original_language', 'popularity', 'revenue', 'runtime', 'release_year',
       'orig_lang_cd', 'collection', 'website', 'genre_name', 'g_animation',
       'g_comedy', 'g_family', 'g_adventure', 'g_fantasy', 'g_romance',
       'g_drama', 'g_action', 'g_crime', 'g_thriller', 'g_horror', 'g_history',
       'g_mystery', 'g_war', 'g_foreign', 'g_music', 'g_do

In [19]:
joined_tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   imdb_id                70529 non-null  object
 1   title                  70529 non-null  object
 2   year                   70529 non-null  object
 3   duration               70529 non-null  object
 4   country                70465 non-null  object
 5   language               69913 non-null  object
 6   avg_vote               70529 non-null  object
 7   votes                  70529 non-null  object
 8   budget                 21107 non-null  object
 9   usa_gross_income       14632 non-null  object
 10  worlwide_gross_income  30375 non-null  object
 11  metascore              12858 non-null  object
 12  reviews_from_users     63624 non-null  object
 13  reviews_from_critics   60360 non-null  object
 14  release_year           70529 non-null  object
 15  genre_list         

In [17]:
query2 = '''ALTER TABLE joined_tables DROP COLUMN ;'''
pd.read_sql_query(query2, con=engine, chunksize = 20)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "'genre_list'"
LINE 1: ALTER TABLE joined_tables DROP COLUMN 'genre_list';
                                              ^

[SQL: ALTER TABLE joined_tables DROP COLUMN 'genre_list';]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [18]:
joined_tables.columns

Index(['imdb_id', 'title', 'year', 'duration', 'country', 'language',
       'avg_vote', 'votes', 'budget', 'usa_gross_income',
       'worlwide_gross_income', 'metascore', 'reviews_from_users',
       'reviews_from_critics', 'release_year', 'genre_list', 'g_action',
       'g_adult', 'g_adventure', 'g_animation', 'g_biography', 'g_comedy',
       'g_crime', 'g_documentary', 'g_drama', 'g_family', 'g_fantasy',
       'g_history', 'g_horror', 'g_music', 'g_musical', 'g_mystery', 'g_news',
       'g_reality_tv', 'g_romance', 'g_sci_fi', 'g_sport', 'g_thriller',
       'g_war', 'g_western', 'director_list', 'budget', 'imdb_id',
       'original_language', 'popularity', 'revenue', 'runtime', 'release_year',
       'orig_lang_cd', 'collection', 'website', 'genre_name', 'g_animation',
       'g_comedy', 'g_family', 'g_adventure', 'g_fantasy', 'g_romance',
       'g_drama', 'g_action', 'g_crime', 'g_thriller', 'g_horror', 'g_history',
       'g_mystery', 'g_war', 'g_foreign', 'g_music', 'g_do

In [8]:
#write the joined_tables to postres
joined_tables.to_sql("joined_tables", engine, if_exists = 'append', index = False, chunksize = 20)

In [22]:
# below is another method of creating connection between AWS and postgres but was not used
# import data from AWS RDS database
#def connect():
    
# Set up a connection to the postgres server.
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
conn = psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()
    

#return conn, cursor


Connected!


In [23]:
# read file from AWS - was not used but a similar method
movie_test_df = pd.read_sql_query('select * from IMDb_main', conn)
movie_test_df.head()


,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_mystery,g_news,g_reality_tv,g_romance,g_sci_fi,g_sport,g_thriller,g_war,g_western,director_list
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,None,None,...,0,0,0,1,0,0,0,0,0,['Raymond Longford']
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,None,None,...,0,0,0,0,0,0,0,0,0,['Arthur Robison']
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,None,None,...,0,0,0,1,0,0,0,0,0,['Jean Epstein']
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,None,...,0,0,0,0,0,0,0,0,0,['Cecil B. DeMille']
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,None,None,...,0,0,0,0,0,0,0,0,0,"['Edward Sedgwick', ' Buster Keaton']"


In [24]:

joined_tables2_df = pd.read_sql_query('select * from joined_tables2', conn)
joined_tables2_df.head()

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_war,g_western,director_list,original_language,popularity,revenue,runtime,orig_lang_cd,collection,website
0,None,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,NaN,None,...,0,0,['Raymond Longford'],None,NaN,NaN,NaN,None,None,None
1,None,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,NaN,None,...,0,0,['Arthur Robison'],None,NaN,NaN,NaN,None,None,None
2,None,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,NaN,None,...,0,0,['Jean Epstein'],None,NaN,NaN,NaN,None,None,None
3,None,Il re dei re,1927.0,160,USA,English,7.2,1890,NaN,None,...,0,0,['Cecil B. DeMille'],None,NaN,NaN,NaN,None,None,None
4,None,Il cameraman,1928.0,76,USA,English,8.1,10101,NaN,None,...,0,0,"['Edward Sedgwick', ' Buster Keaton']",None,NaN,NaN,NaN,None,None,None
